In [3]:
!pip3 install gdown
import pandas as pd
import gdown

url = 'https://drive.google.com/uc?id=1-WZKE5xHw-3m_SL_PtOgwkzdFROIWqih'
output = "reviews.csv"
gdown.download(url, output, quiet=True)


[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


'reviews.csv'

In [4]:
raw_df = pd.read_csv(output)
raw_df = raw_df[raw_df["Score"] > 0]
top1000df = raw_df[:1000]

In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger') 
from nltk.corpus import wordnet

from collections import defaultdict
lemmatizer = WordNetLemmatizer()
tag_map = defaultdict(lambda : wordnet.NOUN)
tag_map['J'] = wordnet.ADJ
tag_map['V'] = wordnet.VERB
tag_map['R'] = wordnet.ADV
def preprocess_with_lemmatization(document): 
    tokens = nltk.word_tokenize(document.lower()) 
    tagged_tokens = nltk.pos_tag(tokens) #get pos of each token
    lemmas = [] 
    for token, pos in tagged_tokens:    #for each token; 
        lemmatizer_tag = tag_map[pos[0]]    #change the pos to wordnet pos; with Noun as the default value
        lemma = lemmatizer.lemmatize(token, pos=lemmatizer_tag) #lemmatize given the token and the lemmatizer tag
        lemmas.append(lemma)
    return lemmas

In [15]:
data_set = raw_df[:10000]
training_data = data_set[:9000]
testing_data = data_set[9000:]

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

#training data
training_vectors = vectorizer.fit_transform(training_data["Text"])
training_score = training_data["Score"]

#testing data
testing_vectors = vectorizer.transform(testing_data["Text"])
testing_score = testing_data["Score"]

first_training_vector = training_vectors.toarray()[0]
print(first_training_vector)
print(len(first_training_vector))

first_testing_vector = testing_vectors.toarray()[0]
print(first_testing_vector)
print(len(first_testing_vector))

[0 0 0 ... 0 0 0]
18141
[0 0 0 ... 0 0 0]
18141


In [17]:
from sklearn.linear_model import LogisticRegression

lr_classifier = LogisticRegression()
lr_classifier.fit(training_vectors, training_score)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [18]:
lr_classifier.score(testing_vectors, testing_score)

0.771